In [2]:
pip install fair-esm torch pandas tqdm

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import torch
import esm
from tqdm import tqdm
import os
import csv

# Use MPS on Apple Silicon if available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"🖥️  Using device: {device}")

# Load ESM-2 model
print("⬇️  Loading ESM-2 model (650M)...")
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model = model.to(device)
model.eval()
print("✅ Model loaded")

# Process both datasets
for dataset in ["algpred2_train.csv", "algpred2_test.csv"]:
    base = os.path.splitext(os.path.basename(dataset))[0]
    df = pd.read_csv(dataset)
    sequences = list(df["sequence"])
    labels = list(df["label"])
    ids = list(df["id"])

    temp_file = f"{base}_esm2_embeddings_temp.csv"
    final_file = f"{base}_esm2_embeddings.csv"

    # Track already processed IDs
    if os.path.exists(temp_file):
        processed_ids = set(pd.read_csv(temp_file, usecols=["id"])["id"])
        print(f"🔁 Resuming from {temp_file} — {len(processed_ids)} entries already processed.")
    else:
        processed_ids = set()

    # Filter out processed entries
    remaining_data = [(ids[i], sequences[i], labels[i]) for i in range(len(ids)) if ids[i] not in processed_ids]

    print(f"⚙️  Extracting embeddings for {dataset}... ({len(remaining_data)} remaining)")

    batch_size = 4
    feature_dim = 1280
    write_header = not os.path.exists(temp_file)
    fieldnames = ["id", "label"] + [f"f{k}" for k in range(feature_dim)]

    with open(temp_file, mode="a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if write_header:
            writer.writeheader()

        for i in tqdm(range(0, len(remaining_data), batch_size)):
            batch = remaining_data[i:i + batch_size]
            batch_ids = [x[0] for x in batch]
            batch_seqs = [x[1] for x in batch]
            batch_labels = [x[2] for x in batch]

            batch_data = [(batch_ids[j], batch_seqs[j]) for j in range(len(batch_seqs))]
            _, _, batch_tokens = batch_converter(batch_data)
            batch_tokens = batch_tokens.to(device)

            with torch.no_grad():
                outputs = model(batch_tokens, repr_layers=[33])
                token_representations = outputs["representations"][33]

            rows = []
            for j, (_, seq) in enumerate(batch_data):
                # Mean over amino acid tokens (1:len+1)
                representation = token_representations[j, 1:len(seq) + 1].mean(0)
                entry = {
                    "id": batch_ids[j],
                    "label": batch_labels[j],
                }
                for k in range(feature_dim):
                    entry[f"f{k}"] = representation[k].item()
                rows.append(entry)

            writer.writerows(rows)

    os.replace(temp_file, final_file)
    print(f"✅ Final embeddings saved to '{final_file}'\n")


🖥️  Using device: mps
⬇️  Loading ESM-2 model (650M)...
✅ Model loaded
🔁 Resuming from algpred2_train_esm2_embeddings_temp.csv — 1934 entries already processed.
⚙️  Extracting embeddings for algpred2_train.csv... (14186 remaining)


  1%|          | 27/3547 [01:27<3:10:06,  3.24s/it]


KeyboardInterrupt: 

In [ ]:
import pandas as pd

df = pd.read_csv("algpred2_train.csv")
print(df.head())
print(df.shape)  # Rows, features

     id                                           sequence  label
0  P_13  MGKPFTLSLSSLCLLLLSSACFAISSSKLNECQLNNLNALEPDHRV...      1
1  P_14  MGVFTFEDEINSPVAPATLYKALVTDADNVIPKALDSFKSVENVEG...      1
2  P_17  MAEDEDNQQGQGEGLKYLGFVQDAATYAVTTFSNVYLFAKDKSGPL...      1
3  P_46  MGVFNYEVETPSVISAARLFKSYVLDGDKLIPKVAPQAITSVENVG...      1
4  P_47  MGVFNYEVETPSVIPAARLFKSYVLDGDKLIPKVAPQAITSVENVE...      1
(16120, 3)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.dummy import DummyClassifier
import xgboost as xgb
import random

# ====================================
# Step 1: Load Data (Proper Separation)
# ====================================
df_train = pd.read_csv("algpred2_train_esm2_embeddings.csv")
df_test = pd.read_csv("algpred2_test_esm2_embeddings.csv")

feature_cols = [f"f{i}" for i in range(1280)]  # Adjust if using larger model
X_temp = df_train[feature_cols].values
y_temp = df_train["label"].values

X_test = df_test[feature_cols].values
y_test = df_test["label"].values

print(f"📁 Train+Val size: {X_temp.shape}, Test size: {X_test.shape}")


# ====================================
# Step 3: Dummy Classifier Baseline (on Train+Val)
# ====================================
print("\n📉 DummyClassifier (Stratified) on Train+Val:\n")
dummy = DummyClassifier(strategy="stratified", random_state=42)
dummy_aucs = []
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, val_idx in cv.split(X_temp, y_temp):
    dummy.fit(X_temp[train_idx], y_temp[train_idx])
    y_dummy_proba = dummy.predict_proba(X_temp[val_idx])[:, 1]
    auc = roc_auc_score(y_temp[val_idx], y_dummy_proba)
    dummy_aucs.append(auc)

print(f"📊 Dummy ROC-AUC: {np.mean(dummy_aucs):.4f} ± {np.std(dummy_aucs):.4f}")

# ====================================
# Step 4: Cross-Validation on Train+Val (XGBoost)
# ====================================
print("\n🚀 5-Fold Cross-Validation (XGBoost) on Train+Val...\n")
xgb_aucs = []

for fold, (train_idx, val_idx) in enumerate(cv.split(X_temp, y_temp)):
    X_train, X_val = X_temp[train_idx], X_temp[val_idx]
    y_train, y_val = y_temp[train_idx], y_temp[val_idx]

    clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_val)
    y_proba = clf.predict_proba(X_val)[:, 1]

    auc = roc_auc_score(y_val, y_proba)
    xgb_aucs.append(auc)

    print(f"📂 Fold {fold+1} AUC: {auc:.4f}")
    print(classification_report(y_val, y_pred, digits=4))
    print("------")

print(f"\n✅ Mean CV ROC-AUC: {np.mean(xgb_aucs):.4f} ± {np.std(xgb_aucs):.4f}")

# ====================================
# Step 5: Final Test Set Evaluation
# ====================================
print("\n🔒 Final Evaluation on Hold-Out Test Set...\n")
clf_final = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)
clf_final.fit(X_temp, y_temp)

y_test_pred = clf_final.predict(X_test)
y_test_proba = clf_final.predict_proba(X_test)[:, 1]

test_auc = roc_auc_score(y_test, y_test_proba)
print(classification_report(y_test, y_test_pred, digits=4))
print(f"🎯 Final Test ROC-AUC: {test_auc:.4f}")

# ====================================
# Step 6: Y-Scrambling Control
# ====================================
print("\n🧪 Y-Scrambling (sanity check) on Train+Val...\n")
y_temp_scrambled = y_temp.copy()
random.seed(42)
random.shuffle(y_temp_scrambled)

scrambled_aucs = []
for train_idx, val_idx in cv.split(X_temp, y_temp_scrambled):
    X_train, X_val = X_temp[train_idx], X_temp[val_idx]
    y_train, y_val = y_temp_scrambled[train_idx], y_temp_scrambled[val_idx]

    clf_scrambled = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)
    clf_scrambled.fit(X_train, y_train)
    y_proba_scrambled = clf_scrambled.predict_proba(X_val)[:, 1]

    auc = roc_auc_score(y_val, y_proba_scrambled)
    scrambled_aucs.append(auc)

print(f"🔀 Y-Scrambled ROC-AUC: {np.mean(scrambled_aucs):.4f} ± {np.std(scrambled_aucs):.4f}")
print("👉 This should be near 0.5 if your real model learned something.")


📁 Train+Val size: (16120, 1280), Test size: (4030, 1280)

📉 DummyClassifier (Stratified) on Train+Val:

📊 Dummy ROC-AUC: 0.4991 ± 0.0000

🚀 5-Fold Cross-Validation (XGBoost) on Train+Val...



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:28:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


📂 Fold 1 AUC: 0.9977
              precision    recall  f1-score   support

           0     0.9900    0.9845    0.9872      1612
           1     0.9846    0.9901    0.9873      1612

    accuracy                         0.9873      3224
   macro avg     0.9873    0.9873    0.9873      3224
weighted avg     0.9873    0.9873    0.9873      3224

------


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:29:16] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


📂 Fold 2 AUC: 0.9982
              precision    recall  f1-score   support

           0     0.9863    0.9845    0.9854      1612
           1     0.9845    0.9864    0.9854      1612

    accuracy                         0.9854      3224
   macro avg     0.9854    0.9854    0.9854      3224
weighted avg     0.9854    0.9854    0.9854      3224

------


...

📂 Fold 5 AUC: 0.9988
              precision    recall  f1-score   support

           0     0.9882    0.9864    0.9873      1612
           1     0.9864    0.9882    0.9873      1612

    accuracy                         0.9873      3224
   macro avg     0.9873    0.9873    0.9873      3224
weighted avg     0.9873    0.9873    0.9873      3224

------

✅ Mean CV ROC-AUC: 0.9983 ± 0.0004

🔒 Final Evaluation on Hold-Out Test Set...



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:31:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0     0.7800    0.9906    0.8728      2015
           1     0.9871    0.7206    0.8330      2015

    accuracy                         0.8556      4030
   macro avg     0.8835    0.8556    0.8529      4030
weighted avg     0.8835    0.8556    0.8529      4030

🎯 Final Test ROC-AUC: 0.9793

🧪 Y-Scrambling (sanity check) on Train+Val...



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:32:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:33:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:33:54] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:34:43] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:35:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

🔀 Y-Scrambled ROC-AUC: 0.4913 ± 0.0076
👉 This should be near 0.5 if your real model learned something.
